# Increasing the Maximum Message Size for gRPC


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)


# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

An example start command using the kvm2 driver would look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


## Start seldon-core

Install the custom resource definition

In [6]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Dec 12 08:18:58 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          1s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        1s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s


NOTES:
NOTES: TODO




In [7]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Wed Dec 12 08:19:02 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          1s
seldon-core-seldon-apiserver        1        1        1           0          1s
seldon-core-seldon-cluster-manager  1        1        1           0          1s
seldon-core-redis                   1        1        1           0          1s

==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-ambassador-56cb8fc595-n9hd6              0/1    ContainerCreating  0         1s
seldon-core-seldon-apiserver-9cc778f6f-4l2nx         0/1    ContainerCreating  0         1s
seldon-core-seldon-cluster-manager-67467f6d5d-w62gw  0/1    ContainerCreating  0         1s
seldon-core-redis-78bb97f8c4-npkfr                   0/1    ContainerCreating  

Check all services are running before proceeding.

In [8]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward to API Gateway**

REST:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

GRPC:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8005:8080
```

Install gRPC modules for the prediction protos.

In [9]:
!cp ../proto/prediction.proto ./proto
!cd ../proto/tensorflow && make create_protos
!cp -vr ../proto/tensorflow/tensorflow .
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

make: Nothing to be done for 'create_protos'.
'../proto/tensorflow/tensorflow/core/framework/types.proto' -> './tensorflow/core/framework/types.proto'
'../proto/tensorflow/tensorflow/core/framework/resource_handle.proto' -> './tensorflow/core/framework/resource_handle.proto'
'../proto/tensorflow/tensorflow/core/framework/tensor_shape.proto' -> './tensorflow/core/framework/tensor_shape.proto'
'../proto/tensorflow/tensorflow/core/framework/tensor.proto' -> './tensorflow/core/framework/tensor.proto'


In [10]:
!pygmentize resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                     

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [11]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [12]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-7cd068f replicas:1 replicasAvailable:1]] state:Available]

## Get predictions - no grpx max message size

In [13]:
from seldon_utils import *
API_GATEWAY_REST="localhost:8003"
API_GATEWAY_GRPC="localhost:8004"
API_AMBASSADOR="localhost:8005"

Send a small request which should suceed.

In [14]:
grpc_request_api_gateway('oauth-key','oauth-secret',None,API_GATEWAY_REST,API_GATEWAY_GRPC,10)

{"access_token":"5bc7258b-ea1a-4497-be2f-9ec6c6b3d5fc","token_type":"bearer","expires_in":43199,"scope":"read write"}


meta {
  puid: "9qopglcod385bsqc2c7dja5h7f"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08778871473756068
  }
}

Test via Ambassador

In [15]:
grpc_request_ambassador("seldon-model",None,API_AMBASSADOR,data_size=10,rows=1)

meta {
  puid: "neqqlovjclhkbavpikd3kph94u"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07680694990557581
  }
}

Send a large request which will be above the default gRPC message size and will fail.

In [16]:
grpc_request_api_gateway('oauth-key','oauth-secret',None,API_GATEWAY_REST,API_GATEWAY_GRPC,1000000)

{"access_token":"5bc7258b-ea1a-4497-be2f-9ec6c6b3d5fc","token_type":"bearer","expires_in":43190,"scope":"read write"}


_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.CANCELLED
	details = "Received RST_STREAM with error code 8"
	debug_error_string = "{"created":"@1544602950.418861744","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"Received RST_STREAM with error code 8","grpc_status":1}"
>

In [17]:
grpc_request_ambassador("seldon-model",None,API_AMBASSADOR,data_size=1000000,rows=1)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "upstream connect error or disconnect/reset before headers"
	debug_error_string = "{"created":"@1544602960.064851081","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"upstream connect error or disconnect/reset before headers","grpc_status":14}"
>

In [18]:
!kubectl delete -f resources/model.json

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


In [19]:
!helm delete seldon-core --purge

release "seldon-core" deleted


# Allowing larger gRPC messages

Recreate seldon-core with extra annotation for the API Gateway

In [20]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true \
    -f grpc_size_values.yaml

NAME:   seldon-core
LAST DEPLOYED: Wed Dec 12 08:23:09 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1        1        1           0          0s
seldon-core-redis                   1        1        1           0          0s

==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-ambassador-56cb8fc595-8w6m2              0/1    ContainerCreating  0         0s
seldon-core-seldon-apiserver-cfcbccf69-wkdkb         0/1    ContainerCreating  0         0s
seldon-core-seldon-cluster-manager-67467f6d5d-lqn5s  0/1    ContainerCreating  0         0s
seldon-core-redis-78bb97f8c4-rn5rm                   0/1    ContainerCreating  

Wait for seldon core deployment to be running

In [21]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


Now we change our SeldonDeployment to include a annotation for max grpx message size.

In [22]:
!pygmentize resources/model_grpc_size.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "annotations": {
	    "seldon.io/grpc-max-message-size":"10000000"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_grpc:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                     

In [23]:
!kubectl create -f resources/model_grpc_size.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


### **Make sure you rerun the port forwards above as a new seldon core API gateway and Ambassador will have started**

In [24]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:1 name:test-deployment-grpc-size-fd60a01 replicas:1]] state:Available]

Send a large message. This time it should succeed.

In [25]:
grpc_request_api_gateway('oauth-key','oauth-secret',None,API_GATEWAY_REST,API_GATEWAY_GRPC,data_size=1000000,rows=1)

{"access_token":"30e4a7f3-4c3c-4f25-9a6b-545046d6eb90","token_type":"bearer","expires_in":43199,"scope":"read write"}


meta {
  puid: "vgtrbp6a8h8l2k2mapq0n4ufet"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_grpc:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08189130647209598
  }
}

Send a request via ambassador. This should also succeed.

In [26]:
grpc_request_ambassador("seldon-model",None,API_AMBASSADOR,data_size=1000000,rows=1)

meta {
  puid: "8huu5j498384tt96p2j8picej5"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_grpc:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0818420273278805
  }
}